Part I

1-)

Layer-> Output Volume Shape ->  Number of Parameters

Input-> 127x127x4 ->  0

CONV3-10-> 125x125x4 -> 360

POOL-3-> 62x62x4 -> 0

CONV3-10-> 60x60x4 -> 360

POOL-2 -> 30x30x4 -> 0

FC-20-> 20 -> 72000

FC-10 -> 10 -> 200




2-)
Output1 = 1
Output2 = 0
Output3 = 1
Output4 = 0


3-)
I-)
Input -> 1st hidden layer : 5x3 = 15
1st hidden layer -> 2nd hidden layer : 3x5 = 15
2nd hidden layer -> Output : 2 x 3 = 6
sum -> 36

II-) Already fully connected. So the answer remains same

III-) Network is fully connected. So all neurons connected the neurons which places in the previous layer. So each nodes take information from all neurons placed in previous layes. In forward and back-propagation, all connections get a value and calculated in propagation. It will be computationally complex    


# Part II

1.   Apply gray scale to each image
1.   Change image name by their class


1.   Create a MLNN with n hidden layer
1.   Use Sum Of The Negative Log-Likelihood  Loss Function
1.   Update parameters with gradient descent
1.   Try with different learning rate and batch sizes




1.   Determine number of units in your convolutional layers and fully connected layers.
2.   Determine the convolutional layer’s width, height, and depth parameters.
2.   Determine batch size as you learned in class
2.   Determine a learning rate for your gradient descent method.
2.   Start with a learning rate value and after each epoch, reduce the learning rate by multiplying it by a decay rate.
2.   Save your trained models to use later in test time


  







# Import Libraries

In [30]:
!pip install --upgrade python
!nvidia-smi

!pip install tensorflow tensorflow-gpu opencv-python matplotlib
!pip list
!pip install tensorflow-metal
!pip install tensorflow-macos

ERROR: Could not find a version that satisfies the requirement python (from versions: none)
ERROR: No matching distribution found for python
/bin/bash: line 1: nvidia-smi: command not found
  Using cached tensorflow-gpu-2.12.0.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations     

In [31]:
from PIL import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import os
import shutil
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout,BatchNormalization
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.callbacks import ModelCheckpoint
import random
import time
import csv

import cv2
import imghdr


from google.colab import drive


In [32]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
tf.config.list_physical_devices('GPU')


drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CONSTANTS

In [33]:

RESIZE_SHAPE = (32,32)

# Specify the path to the directory you want to change to
FOLDER_PATH = "/content/drive/MyDrive/Colab Notebooks/MLlast"

# Get Data Ready for NN

In [34]:

os.chdir(FOLDER_PATH)


def change_path(path : str ):
  """
  Loging into folder that placed in given path
  """
  new_path = FOLDER_PATH + path

    # Check if the folder exists
  if not os.path.exists(new_path):
      # If not, create the folder
      os.makedirs(new_path)
      print(f"Folder '{new_path}' created.")

  # Change the current working directory to 5the new directory
  os.chdir(new_path)

  # Get the current directory (the directory where your Python script is located)
  current_directory = os.getcwd()
  print(current_directory)

  files = os.listdir(current_directory)




  return new_path

change_path("")


/content/drive/MyDrive/Colab Notebooks/MLlast


'/content/drive/MyDrive/Colab Notebooks/MLlast'

In [35]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

train_df = train_df.drop(columns =  ["gender","accent","text"])
test_df = test_df.drop(columns =  ["gender","accent","text"])


In [36]:
def grayscale_image(image_path, output_path):
    # Load the image
    image = Image.open(image_path)

    # Convert to grayscale
    grayscale_image = image.convert('L')

    # Save the grayscale image with the same name in the same folder
    grayscale_image.save(output_path)




In [37]:
# Iterate over unique age values and create directories

def organize_files(to_folder : str, df : pd.core.frame.DataFrame):
  # to_folder can be /train or /test
  """
  Group files to folder
  """
  base_directory = change_path(to_folder)

  for age_group in df['age'].unique():
      age_directory = os.path.join(base_directory, age_group)
      os.makedirs(age_directory, exist_ok=True)

  # Move files to respective age directories
  for index, row in df.iterrows():
    try:

      source_file = row['filename']
      age_group = row['age']
      destination_directory = os.path.join(base_directory, age_group)
      destination_file = os.path.join(destination_directory, source_file)

      # Move the file
      shutil.move(source_file, destination_file)
    except:
      pass
  print("Files organized into age-specific directories.")



In [38]:
def number_of_files(folder_path : str):
  """
  Returns number of files in given folder
  """
  # Get the list of files in the folder
  files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

  # Get the number of files
  num_files = len(files)
  return num_files
  print(f'Number of files in the folder: {num_files}')


In [39]:
def find_number_of_total_files_in_folder(path : str):
  """
  Number of total files in given folder
  """
  change_path(path)
  folders = list(test_df["age"].unique())
  length =  0

  for i in folders:
    length += number_of_files(i)
  print("Total File : ",length)

  return length


## Copy Folder for Gray Scaling

In [40]:

def copy_folder(source_folder, destination_folder):
    try:
        shutil.copytree(source_folder, destination_folder)
        print(f"Folder '{source_folder}' copied to '{destination_folder}'.")
    except Exception as e:
        print(f"An error occurred: {e}")

def copy_both():
  source_folder = FOLDER_PATH+"/test"
  destination_folder =  FOLDER_PATH+"/gray_test"

  copy_folder(source_folder, destination_folder)


  source_folder = FOLDER_PATH+"/train"
  destination_folder =  FOLDER_PATH+"/gray_train"

  copy_folder(source_folder, destination_folder)

# Dont run this pipeline if you have already folders named like that
#copy_both()

## Gray Scale Images and Add Them Another Folder

In [41]:
if find_number_of_total_files_in_folder("/gray_test") < 2000:
  # If images not gray scaled, then scale them

  change_path("/gray_test")
  current_directory = os.getcwd()
  print(current_directory)
  # Iterate over files in the current directory
  for filename in os.listdir(current_directory):
      if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more file extensions if needed
          # Generate output path with 'grayscale_' prefix
          output_path = os.path.join(current_directory, f'{filename}')

          # Convert to grayscale and save
          grayscale_image(filename, output_path)

  print("Grayscale conversion completed.")

  organize_files("/gray_test",test_df)


  print("Organization completed successfully!!!!")

else:
  print("Images are already gray scaled and organized")

/content/drive/MyDrive/Colab Notebooks/MLlast/gray_test
Total File :  2350
Images are already gray scaled and organized


## Gray Scale Images and Add Them Another Folder

In [42]:
if find_number_of_total_files_in_folder("/gray_train") < 9000:
  # If images not gray scaled, then scale them

  change_path("/gray_train")

  current_directory = os.getcwd()
  print(current_directory)
  # Iterate over files in the current directory
  for filename in os.listdir(current_directory):
      if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more file extensions if needed
          # Generate output path with 'grayscale_' prefix
          output_path = os.path.join(current_directory, f'{filename}')

          # Convert to grayscale and save
          grayscale_image(filename, output_path)

  print("Grayscale conversion completed.")

  organize_files("/gray_train",train_df)

  print("Organization completed successfully!!!!")

else:
  print("Images are already gray scaled and organized")

/content/drive/MyDrive/Colab Notebooks/MLlast/gray_train
Total File :  9600
Images are already gray scaled and organized


## Just Gray Scaling

In [43]:
def gray_scale_conversion():
  """
  Gray Scale all images in test and train folder and add them to test and test
  """
  source_folder = FOLDER_PATH+"/test"
  destination_folder =  FOLDER_PATH+"/test"

  copy_folder(source_folder, destination_folder)


  source_folder = FOLDER_PATH+"/train"
  destination_folder =  FOLDER_PATH+"/train"

  copy_folder(source_folder, destination_folder)

  change_path("/train")

  current_directory = os.getcwd()
  # Iterate over files in the current directory
  for filename in os.listdir(current_directory):
      if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more file extensions if needed
          # Generate output path with 'grayscale_' prefix
          output_path = os.path.join(current_directory, f'{filename}')

          # Convert to grayscale and save
          grayscale_image(filename, output_path)


  change_path("/test")
  current_directory = os.getcwd()
  # Iterate over files in the current directory
  for filename in os.listdir(current_directory):
      if filename.endswith(('.jpg', '.jpeg', '.png')):  # Add more file extensions if needed
          # Generate output path with 'grayscale_' prefix
          output_path = os.path.join(current_directory, f'{filename}')

          # Convert to grayscale and save
          grayscale_image(filename, output_path)

  print("Grayscale conversion completed.")

#gray_scale_conversion()

In [44]:
change_path("")


/content/drive/MyDrive/Colab Notebooks/MLlast


'/content/drive/MyDrive/Colab Notebooks/MLlast'

# MLP

## Vectorize images

In [45]:
def vectorizer(image_name):
  # Reshaping image
  img = Image.open(image_name).resize(RESIZE_SHAPE)

  # Normalizing image
  img_array = np.array(img)/255
  return img_array.flatten()

## Train Test Split

In [ ]:
test_name_list = os.listdir(change_path("/gray_test2"))

vectorized_test_images = [] # It will hold train images which gray scaled and normalized

for index , test_names in enumerate(test_name_list):
  try:
    vectorized_test_images.append(vectorizer(test_names))
    print(index , end = " ")
  except:
    vectorized_test_images.append(vectorizer(test_name_list[index-1]))
    print("Error in : ",str(index), "th index")

/content/drive/MyDrive/Colab Notebooks/MLlast/gray_test2
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 2

In [ ]:
train_name_list = os.listdir(change_path("/gray_train2"))
vectorized_train_images = [] # It will hold train images which gray scaled and normalized

for index, train_names in enumerate(train_name_list):
  try:
    vectorized_train_images.append(vectorizer(train_names))
    print(index , end = " ")
  except:
    vectorized_train_images.append(vectorizer(train_name_list[index-1]))
    print("Error in : ",str(index), "th index")

/content/drive/MyDrive/Colab Notebooks/MLlast/gray_train2
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 

In [ ]:
test_labels = []

for index, file_name in enumerate(test_name_list):
  test_labels.append(test_df.loc[test_df['filename'] == file_name]["age"].values[0])
  print(index , end = " ")




0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [ ]:
train_labels = []

for index, file_name in enumerate(train_name_list):
  train_labels.append(train_df.loc[train_df['filename'] == file_name]["age"].values[0])
  print(index , end = " ")




0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

## Functions

In [50]:
## Activation Functions

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)

def ReLU(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return np.where(z > 0, 1, 0)

def tanh(x):
    return np.tanh(x)

def tanh_derivative(x):
    return 1.0 - np.tanh(x)**2

def negative_log_likelihood(y_true, y_pred):
    epsilon = 1e-15
    y_pred = tf.clip_by_value(y_pred, epsilon, 1 - epsilon)
    return -tf.reduce_sum(y_true * tf.math.log(y_pred)) / len(y_true)

In [51]:
# Constants

INPUT_SIZE = RESIZE_SHAPE[0] *  RESIZE_SHAPE[1]
OUTPUT_SIZE = 6
loss = 0


In [53]:
"""
To be honest, I could handle CNN part by myself and of course helps of GPT but I couldn't make MLP by myself , so I decided to take ready code from GitHub and adapting it to our assignment
"""

def initialize_weights_and_biases(num_layers, sizes):
    """
    This initializes weights and biases by assining biases 0 and random numbers to weights

    ATTENTION : Assign random numbers to weights will change accuracies if number of epoch is low

    """
    weights = [np.random.randn(sizes[i], sizes[i + 1]) for i in range(num_layers + 1)]
    biases = [np.zeros((1, sizes[i + 1])) for i in range(num_layers + 1)]
    return weights, biases



def forward_pass(X, weights, biases, activation_function):
    """
    This makes forwad pass process in neural network by taking inputs and gives them to the activation functions as parameter
    """
    activations = [X]
    forward = []

    for step in range(len(weights)):
        pass1 = X.dot(weights[step]) + biases[step]
        forward.append(pass1)

        activation = activation_function(pass1)
        activations.append(activation)
        X = activation

    return activation, activations, forward

def backward_pass(y, weights, biases, activations, forward, activation_function_derive, learning_rate):
    """
    It calculates errors in network by applying gradient descent algorithm. It takes derivations of functions and appyling back propagation on entire network
    """

    layr_errors = activations[-1] - y

    for step in range(len(weights) - 1, -1, -1):
        d_weights = activations[step].T.dot(layr_errors * activation_function_derive(forward[step]))
        d_bias = np.sum(layr_errors * activation_function_derive(forward[step]), axis = 0, keepdims=True)
        layr_errors = layr_errors.dot(weights[step].T)

        weights[step] -= learning_rate * d_weights
        biases[step] -= learning_rate * d_bias

def calculate_loss(y_true, y_pred, negative_log_likelihood):
    """
    It returns loss function as negative log likelihood
    """
    return negative_log_likelihood(y_true, y_pred)

def fit(X, y, num_layers, sizes, activation_function, activation_function_derive, learning_rate, epochs, batch_size, negative_log_likelihood):
    """
    It makes fit part of neural network by using functions that created above. It is an MLP neural network
    """
    weights, biases = initialize_weights_and_biases(num_layers, sizes)
    loss = 0

    for each_epoch in range(epochs):
        all_losses = []
        for batch_start in range(0, X.shape[0], batch_size):
            batch_end = min(batch_start + batch_size, X.shape[0])

            X_batch = X[batch_start:batch_end]
            y_batch = y[batch_start:batch_end]

            y_pred, activations, forward = forward_pass(X_batch, weights, biases, activation_function)
            batch_loss = calculate_loss(y_batch, y_pred, negative_log_likelihood)
            backward_pass(y_batch, weights, biases, activations, forward, activation_function_derive, learning_rate)

            all_losses.append(batch_loss)

        loss = np.mean(all_losses)

    return weights, biases, loss

def predict(X, weights, biases, activation_function):
    """
    It returns values which predicted in network
    """
    return forward_pass(X, weights, biases, activation_function)[0]


In [ ]:
X_train =  np.array(vectorized_train_images).reshape(9600,INPUT_SIZE)
X_test = np.array(vectorized_test_images).reshape(2400,INPUT_SIZE)
y_train = LabelEncoder().fit_transform(np.array(train_labels)).reshape(-1, 1)
y_test = LabelEncoder().fit_transform(np.array(test_labels)).reshape(-1, 1)


def create_mlp(num_layers, input_size, sizes, output_size, activation_function, activation_function_derive,learning_rate, epochs, batch_size):
  weights, biases, final_loss = fit(X_train, y_train, num_layers, sizes, activation_function, activation_function_derive, learning_rate, epochs, batch_size, negative_log_likelihood)
  y_pred = predict(X_test, weights, biases, activation_function)
  acc = np.mean(y_pred == y_test)

  return acc


In [ ]:
mlp_df = pd.DataFrame(columns=["Number of Hidden Layers","Input Size","Hidden_Size","Output Size","Activation Function","Derivative Function", "Learning Rate" ,"Epoch","Batch Size" ,"Accuracy"])

# Paramaters
parameter_list= [[1, INPUT_SIZE, [2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128], # Referrence model ,  1 hidden layer, 1024 input size, 2 hidden size, 0.02 learning rate, 128 batch size

                [1, INPUT_SIZE, [2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 32] , # Reduce Batch Size 128 -> 32

                 [1, INPUT_SIZE, [2], OUTPUT_SIZE, tanh, tanh_derivative, 0.02, 10, 128], # Relu -> tanh

                 [1, INPUT_SIZE, [2], OUTPUT_SIZE, sigmoid, sigmoid_derivative, 0.02, 10, 128], # Relu -> sigmoid

                 [1, INPUT_SIZE, [2], OUTPUT_SIZE, ReLU, relu_derivative, 0.005, 10, 128], # Learning Rate 0.02 -> 0.05

                 [1, INPUT_SIZE, [2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 100, 128], # Epoch  10 -> 100

                 [0, INPUT_SIZE, [], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128], # Same thing with 0 hidden layers

                [0, INPUT_SIZE, [], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 32] , # Reduce Batch Size 128 -> 32

                 [0, INPUT_SIZE, [], OUTPUT_SIZE, tanh, tanh_derivative, 0.02, 10, 128], # Relu -> tanh

                 [0, INPUT_SIZE, [], OUTPUT_SIZE, sigmoid, sigmoid_derivative, 0.02, 10, 128], # Relu -> sigmoid

                 [0, INPUT_SIZE, [], OUTPUT_SIZE, ReLU, relu_derivative, 0.005, 10, 128], # Learning Rate 0.02 -> 0.05

                 [0, INPUT_SIZE, [], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 100, 128], # Epoch  10 -> 100

                 [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128], # Same thing with 2 hidden layers with 2 hidden size

                [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 32] , # Reduce Batch Size 128 -> 32

                 [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, tanh, tanh_derivative, 0.02, 10, 128], # Relu -> tanh

                 [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, sigmoid, sigmoid_derivative, 0.02, 10, 128], # Relu -> sigmoid

                 [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, ReLU, relu_derivative, 0.005, 10, 128], # Learning Rate 0.02 -> 0.05

                 [2, INPUT_SIZE, [2,2], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 100, 128],  # Epoch  10 -> 100

                 [1, INPUT_SIZE, [INPUT_SIZE], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128], # 1024 hidden sizes with 1 layer

                 [2, INPUT_SIZE, [INPUT_SIZE,INPUT_SIZE], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128], # 1024 hidden sizes with 2 layer

                 [3, INPUT_SIZE, [INPUT_SIZE,INPUT_SIZE,INPUT_SIZE], OUTPUT_SIZE, ReLU, relu_derivative, 0.02, 10, 128]] # 1024 hidden sizes with 2 layer






In [ ]:
for parameter in parameter_list:
  new_row = []
  new_row.extend([parameter[0],parameter[1],parameter[2],parameter[3],parameter[4].__name__,parameter[5].__name__,parameter[6],parameter[7],parameter[8]])
  acc =  create_mlp(parameter[0],parameter[1],parameter[2],parameter[3],parameter[4],parameter[5],parameter[6],parameter[7],parameter[8])
  acc =  int(acc * 1000) / 1000.0
  new_row.append(acc)
  print(new_row)
  new_row_df = pd.DataFrame([new_row], columns=mlp_df.columns)
  mlp_df = pd.concat([mlp_df, new_row_df], ignore_index=True)

[1, 1024, [1024, 2, 6], 6, 'ReLU', 'relu_derivative', 0.02, 10, 128, 0.083]
[1, 1024, [1024, 2, 6], 6, 'ReLU', 'relu_derivative', 0.02, 10, 32, 0.027]
[1, 1024, [1024, 2, 6], 6, 'tanh', 'tanh_derivative', 0.02, 10, 128, 0.166]
[1, 1024, [1024, 2, 6], 6, 'sigmoid', 'sigmoid_derivative', 0.02, 10, 128, 0.166]
[1, 1024, [1024, 2, 6], 6, 'ReLU', 'relu_derivative', 0.005, 10, 128, 0.055]
[1, 1024, [1024, 2, 6], 6, 'ReLU', 'relu_derivative', 0.02, 100, 128, 0.0]
[0, 1024, [1024, 6], 6, 'ReLU', 'relu_derivative', 0.02, 10, 128, 0.124]
[0, 1024, [1024, 6], 6, 'ReLU', 'relu_derivative', 0.02, 10, 32, 0.166]
[0, 1024, [1024, 6], 6, 'tanh', 'tanh_derivative', 0.02, 10, 128, 0.138]
[0, 1024, [1024, 6], 6, 'sigmoid', 'sigmoid_derivative', 0.02, 10, 128, 0.111]
[0, 1024, [1024, 6], 6, 'ReLU', 'relu_derivative', 0.005, 10, 128, 0.114]
[0, 1024, [1024, 6], 6, 'ReLU', 'relu_derivative', 0.02, 100, 128, 0.159]
[2, 1024, [1024, 2, 2, 6], 6, 'ReLU', 'relu_derivative', 0.02, 10, 128, 0.083]
[2, 1024, [1024

In [ ]:
mlp_df.to_csv("mlp_df.csv", index=False)

## MLP Result

In [ ]:
mlp_df

,Number of Hidden Layers,Input Size,Hidden_Size,Output Size,Activation Function,Derivative Function,Learning Rate,Epoch,Batch Size,Accuracy
0,1,1024,"[1024, 2, 6]",6,ReLU,relu_derivative,0.020,10,128,0.083
1,1,1024,"[1024, 2, 6]",6,ReLU,relu_derivative,0.020,10,32,0.027
2,1,1024,"[1024, 2, 6]",6,tanh,tanh_derivative,0.020,10,128,0.166
3,1,1024,"[1024, 2, 6]",6,sigmoid,sigmoid_derivative,0.020,10,128,0.166
4,1,1024,"[1024, 2, 6]",6,ReLU,relu_derivative,0.005,10,128,0.055
5,1,1024,"[1024, 2, 6]",6,ReLU,relu_derivative,0.020,100,128,0.000
6,0,1024,"[1024, 6]",6,ReLU,relu_derivative,0.020,10,128,0.124
7,0,1024,"[1024, 6]",6,ReLU,relu_derivative,0.020,10,32,0.166
8,0,1024,"[1024, 6]",6,tanh,tanh_derivative,0.020,10,128,0.138
9,0,1024,"[1024, 6]",6,sigmoid,sigmoid_derivative,0.020,10,128,0.111


Firstly I need to say I didn't have a good CPU or GPU, so I decided to take epoch = 10 as a result of that I couldn't get epochs that much. A reason that why my accuracies are too low is, I chose to create my network with different parameters randomly, there was no reason for any parameter. So lets make interpret the result for each feature change. To do that properly, I decided to take a refference model which these parameters


Hidden layer : 1

Batch Size : 128

Activation Function : relu

Derivation of Activation Function : relu_derivative (it cant be changed because activation function is relu)

Learning Rate : 0.02

When apply those, we get accuracy = 0.05

Lets see what will happened changing each feature one by one





## Decreasing Batch Size From 128 -> 32


Number of layer = 0 => 0.083 -> 0.027

Number of layer = 1 => 0.124 -> 0.166

Number of layer = 2 => 0.083 -> 0.055

It changes according to number of layers, but we can say it mosrdecreas


Decreasing batch size is increased stochasticity(smaller batch sizes introduce more randomness and noise into the training process). While this increased stochasticity can sometimes help with generalization, it may also hinder the optimization process, making it harder for the model to converge to a good solution.

Another reason for that may about learning rate. The choice of learning rate is crucial, and smaller batch sizes may require adjustments in the learning rate. If the learning rate is not appropriately tuned, it can lead to poor convergence.

The most acceptable reason can be, if we accept batch size can depend on the specific architecture of the neural network. Some architectures may be more sensitive to batch size variations.



## Changing Activation Function
relu -> tanh -> sigmoid

Number of layer = 0 =>  0.166 -> 0.138 -> 0.111

Number of layer = 1 =>  0.027-> 0.166 -> 0.166

Number of layer = 2 => 0.055 -> 0.166 -> 0.166

tanh and sigmoid functions are better than relu in our model. The reason why sigmoid and tanh has the same accuracies (0.166 for two model)  both sigmoid and tanh functions can suffer from saturation issues, where the output values are pushed to extreme ends of the activation range (0 or 1 for sigmoid, -1 or 1 for tanh). This can lead to vanishing gradients during backpropagation, making it challenging for the network to learn. Another reason that could be a gradient descent variation, the optimization algorithm we are using(stochastic gradient descent), could impact the convergence of the network.Both functions are reaching a similar local minimum, it might result in similar accuracies.


## Learning Rate
0.02 -> 0.005

Number of layer = 0 => 0.124 -> 0.114

Number of layer = 1 => 0.083-> 0.055

Number of layer = 2 => 0.083 -> 0.083

We would expect to have higher accuracy because lower learning rates improves generalization and lowering the learning rate can help stabilize the training process and prevent these issues. But we dont have such result my model be overfitted or it can be caussed initialization problems, poor weight initialization can hinder convergence. (I did this three time and have different values in all times. It there may be a problem about memory)



## Epoch
10 -> 100

Number of layer = 0 => 0.124 -> 0.159

Number of layer = 1 => 0.083-> 0.0

Number of layer = 2 => 0.083 -> 0.138


Normally we would expect to increase in accuracy when to increase number of epoch. And we get the expected reulst except in number of layer 1. The reason might be a memory problem. Maybe it vanishes loss or something else but it isn't an expected situation to get accuracy as 0


## Hidden Layers Sizes

[1024, 1024, 6] -> 0.166

[1024, 1024, 1024, 6] -> 0.166

[1024, 1024, 1024, 1024, 6] -> 0.166

It is an unpextec output actually. There is a lot reason why to have such result. These are memory leak problem, overfitting, or parameter tuning etc. Actually I would expect getter better results in last row ([1024, 1024, 1024, 1024, 6]) but results don't say something like that





# CNN


In [ ]:
# Create an empty DataFrame
parameters_df = pd.DataFrame(columns=['Number of Hidden Layers','Batch Size', 'Activation Function', 'Loss Function','Optimizer',"Normalized & Dropout","Learning Rate","Accuracy"])



In [ ]:
# Paramaters
parameter_list= [[256,"relu",tf.losses.BinaryCrossentropy(), Adam(learning_rate = 0.1)], # Referrence model

                 [64,"relu",tf.losses.BinaryCrossentropy(), Adam(learning_rate = 0.1)], # Reduce Batch Size

                 [256,"tanh",tf.losses.BinaryCrossentropy(), Adam(learning_rate = 0.1)], # Relu -> tanh

                 [256,"sigmoid",tf.losses.BinaryCrossentropy(), Adam(learning_rate =0.1)], # Relu -> sigmoid

                 [256,"relu",tf.losses.MeanSquaredError(), Adam(learning_rate = 0.1)], # BinaryCrossentropy -> MeanSquaredError

                 [256,"relu",tf.losses.BinaryCrossentropy(), RMSprop(learning_rate = 0.1)], # Adam -> RMSprop

                  [256,"relu",tf.losses.BinaryCrossentropy(), Adam(learning_rate =  0.001)], # learning_rate = 0.1 -> learning_rate = 0.001

                [256,"relu",tf.losses.BinaryCrossentropy(), RMSprop(learning_rate =  0.001)]] # RMSprop lr 0.1 -> 0.001

# Stable parameters
epoch_num = 10
"""
Accuracy wasn't change so much when number of epochs increases a little bit (like epoch_num = 10). So I decided to take it 10 to compute results faster
If I could take epoch_num = 100, maybe everything will be different
Epoch 1/10
8/8 [==============================] - 18s 1s/step - loss: -14.9094 - accuracy: 0.1724 - val_loss: -25.3432 - val_accuracy: 0.1705
Epoch 2/10
8/8 [==============================] - 15s 1s/step - loss: -22.7547 - accuracy: 0.1670 - val_loss: -23.3071 - val_accuracy: 0.1591
Epoch 3/10
8/8 [==============================] - 17s 2s/step - loss: -23.0823 - accuracy: 0.1689 - val_loss: -21.4876 - val_accuracy: 0.1761
Epoch 4/10
8/8 [==============================] - 19s 2s/step - loss: -23.0079 - accuracy: 0.1636 - val_loss: -24.9966 - val_accuracy: 0.1562
Epoch 5/10
8/8 [==============================] - 15s 1s/step - loss: -22.9781 - accuracy: 0.1655 - val_loss: -21.6609 - val_accuracy: 0.1648
Epoch 6/10
8/8 [==============================] - 17s 2s/step - loss: -22.9707 - accuracy: 0.1636 - val_loss: -21.6175 - val_accuracy: 0.1960
Epoch 7/10
8/8 [==============================] - 16s 1s/step - loss: -22.6952 - accuracy: 0.1670 - val_loss: -22.6139 - val_accuracy: 0.1790
Epoch 8/10
8/8 [==============================] - 21s 2s/step - loss: -23.0228 - accuracy: 0.1650 - val_loss: -22.2674 - val_accuracy: 0.1903
Epoch 9/10
8/8 [==============================] - 22s 1s/step - loss: -23.0600 - accuracy: 0.1675 - val_loss: -23.8703 - val_accuracy: 0.1534
Epoch 10/10
8/8 [==============================] - 21s 2s/step - loss: -22.1630 - accuracy: 0.1690 - val_loss: -23.6175 - val_accuracy: 0.1644

As we can see, accuracy wasn't change too much. So lets just take number of epochs as 3
"""
filter_size = 16
k_size = (3,3)

## 1 Hidden Layer

In [ ]:
for row_number , each_try in enumerate(parameter_list):
  change_path("")

  batch_size = each_try[0]

  activation_function = each_try[1]

  selected_loss_function = each_try[2]

  selected_optimizer = each_try[3]



  test_data = tf.keras.utils.image_dataset_from_directory('gray_test',batch_size = batch_size, image_size = (256,256),shuffle = True)
  train_data = tf.keras.utils.image_dataset_from_directory('gray_train',batch_size = batch_size , image_size = (256,256),shuffle = True)


  # Scaling Image Values

  train_data = train_data.map(lambda x,y: (x/255, y))
  train_data = test_data.map(lambda x,y: (x/255, y))

  train_size = int(len(train_data)*.8)
  val_size = int(len(train_data)*.2)

  train = train_data.take(train_size)
  val = train_data.skip(train_size).take(val_size)
  test = test_data



  my_model = Sequential()
  # Add a convolution layer and maxpooling
  my_model.add(Conv2D(filter_size,k_size ,  1, activation = activation_function, input_shape = (256,256,3)))
  my_model.add(MaxPooling2D())

  # Flat the network
  my_model.add(Flatten())

  # Add a dense layer
  my_model.add(Dense(256, activation = activation_function))
  my_model.add(Dense(1, activation = activation_function))
  my_model.compile(selected_optimizer, loss= selected_loss_function, metrics=['accuracy'])


  print("Building Model ...")
  my_model.summary()

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
  hist = my_model.fit(train, epochs=epoch_num, validation_data=val, callbacks=[tensorboard_callback])





  lr = 0.1
  if row_number == 6 or row_number == 7:
    lr = 0.001


  # Add elements to the DataFrame using a list
  new_row = [1,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name,"False", str(lr) ,str(hist.history['accuracy'][-1])]
  parameters_df = parameters_df.append(pd.Series(new_row, index=parameters_df.columns), ignore_index=True)


  # Save Model With Random Number
  change_path("/models")
  checkpoint = ModelCheckpoint('imageclassifier.h'+str(random.randint(1,10000)), save_best_only=True)



  print([1,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name])
  break



## 2 Hidden layer

In [ ]:
for row_number , each_try in enumerate(parameter_list):
  change_path("")


  batch_size = each_try[0]

  activation_function = each_try[1]

  selected_loss_function = each_try[2]

  selected_optimizer = each_try[3]


  test_data = tf.keras.utils.image_dataset_from_directory('gray_test',batch_size = batch_size, image_size = (256,256),shuffle = True)
  train_data = tf.keras.utils.image_dataset_from_directory('gray_train',batch_size = batch_size , image_size = (256,256),shuffle = True)


  # Normalizin Image Values

  train_data = train_data.map(lambda x,y: (x/255, y))
  train_data = test_data.map(lambda x,y: (x/255, y))

  train_size = int(len(train_data)*.8)
  val_size = int(len(train_data)*.2)

  train = train_data.take(train_size)
  val = train_data.skip(train_size).take(val_size)
  test = test_data



  my_model = Sequential()
  # Add a convolution layer and maxpooling
  my_model.add(Conv2D(filter_size,k_size ,  1, activation = activation_function, input_shape = (256,256,3)))
  my_model.add(MaxPooling2D())

  # Second Hidden Layer
  my_model.add(Conv2D(filter_size, k_size ,  1, activation = activation_function))
  my_model.add(MaxPooling2D())

  # Flat the network
  my_model.add(Flatten())

  # Add a dense layer
  my_model.add(Dense(256, activation = activation_function))
  my_model.add(Dense(1, activation = activation_function))
  my_model.compile(selected_optimizer, loss= selected_loss_function, metrics=['accuracy'])


  print("Building Model ...")
  my_model.summary()

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
  hist = my_model.fit(train, epochs=epoch_num, validation_data=val, callbacks=[tensorboard_callback])



  lr = 0.1
  if row_number == 6 or row_number == 7:
    lr = 0.001


  # Add elements to the DataFrame using a list
  new_row = [2,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name,"False", str(lr) ,str(hist.history['accuracy'][-1])]
  parameters_df = parameters_df.append(pd.Series(new_row, index=parameters_df.columns), ignore_index=True)


  # Save Model With Random Number
  change_path("/models")
  checkpoint = ModelCheckpoint('imageclassifier.h'+str(random.randint(1,10000)), save_best_only=True)



  print([2,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name])




## Dropout and Normalization to 1 Hidden Layer

In [ ]:
for row_number , each_try in enumerate(parameter_list):
  change_path("")


  batch_size = each_try[0]

  activation_function = each_try[1]

  selected_loss_function = each_try[2]

  selected_optimizer = each_try[3]



  test_data = tf.keras.utils.image_dataset_from_directory('gray_test',batch_size = batch_size, image_size = (256,256),shuffle = True)
  train_data = tf.keras.utils.image_dataset_from_directory('gray_train',batch_size = batch_size , image_size = (256,256),shuffle = True)


  # Normalizin Image Values

  train_data = train_data.map(lambda x,y: (x/255, y))
  train_data = test_data.map(lambda x,y: (x/255, y))

  train_size = int(len(train_data)*.8)
  val_size = int(len(train_data)*.2)

  train = train_data.take(train_size)
  val = train_data.skip(train_size).take(val_size)
  test = test_data



  my_model = Sequential()
  # Add a convolution layer and maxpooling
  my_model.add(Conv2D(filter_size,k_size ,  1, activation = activation_function, input_shape = (256,256,3)))
  my_model.add(MaxPooling2D())

  # Second Hidden Layer
  my_model.add(Conv2D(filter_size, k_size ,  1, activation = activation_function))
  my_model.add(MaxPooling2D())

  # Flat the network
  my_model.add(Flatten())

  # Add a dense layer
  my_model.add(Dense(256, activation = activation_function))
  my_model.add(Dense(1, activation = activation_function))

  # Add a dropout layer with a specified dropout rate (e.g., 0.2)
  my_model.add(Dropout(0.2))

  # Add a normalization layer (Batch Normalization)
  my_model.add(BatchNormalization())
  my_model.compile(selected_optimizer, loss= selected_loss_function, metrics=['accuracy'])


  print("Building Model ...")
  my_model.summary()

  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
  hist = my_model.fit(train, epochs=epoch_num, validation_data=val, callbacks=[tensorboard_callback])

  # Plot result if you want to
  """
  print("Plot Loss Values")

  fig = plt.figure()
  plt.plot(hist.history['loss'], color='teal', label='loss')
  plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
  fig.suptitle('Loss', fontsize=20)
  plt.legend(loc="upper left")
  plt.show()


  print("Plot Accuracy Values")

  fig = plt.figure()
  plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
  plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
  fig.suptitle('Accuracy', fontsize=20)
  plt.legend(loc="upper left")
  plt.show()"""


  lr = 0.1
  if row_number == 6 or row_number == 7:
    lr = 0.001


  # Add elements to the DataFrame using a list
  new_row = [1,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name,"True", str(lr) ,str(hist.history['accuracy'][-1])]
  parameters_df = parameters_df.append(pd.Series(new_row, index=parameters_df.columns), ignore_index=True)


  # Save Model With Random Number
  change_path("/models")
  checkpoint = ModelCheckpoint('imageclassifier.h'+str(random.randint(1,10000)), save_best_only=True)



  print([1,batch_size, activation_function, selected_loss_function.name,selected_optimizer.name])



In [ ]:
# Save df as results.csv
parameters_df.to_csv('results.csv', index=False)

# CNN Result Section

In [ ]:
parameters_df

,Unnamed: 0,Number of Hidden Layers,Batch Size,Activation Function,Loss Function,Optimizer,Normalized & Dropout,Learning Rate,Accuracy
0,0,1,256,relu,binary_crossentropy,Adam,False,0.100,0.168457
1,1,1,64,relu,binary_crossentropy,Adam,False,0.100,0.168750
2,2,1,256,tanh,binary_crossentropy,Adam,False,0.100,0.165527
3,3,1,256,sigmoid,binary_crossentropy,Adam,False,0.100,0.164062
4,4,1,256,relu,mean_squared_error,Adam,False,0.100,0.162598
5,5,1,256,relu,binary_crossentropy,RMSprop,False,0.100,0.163574
6,6,1,256,relu,binary_crossentropy,Adam,False,0.001,0.167969
7,7,1,256,relu,binary_crossentropy,RMSprop,False,0.001,0.165527
8,8,2,256,relu,binary_crossentropy,Adam,False,0.100,0.169922
9,9,2,64,relu,binary_crossentropy,Adam,False,0.100,0.167708


Firstly I need to say I didn't have a good CPU or GPU, so I decided to take epoch < 10 as a result of that I couldn't get different accuracies as much. Another reason that why my accuracies are too low is, I choose to create a random network with different parameters. So I think these accuracies are even count as good accuracy because that randomity. So lets make interpret the result for each feature change. To do that properly, I decided to take a refference model which these parameters


Hidden layer : 1

Batch Size : 256

Activation Function : relu

Note: I have select just one activation function and apply it to entire network
to make comparison easier

Loss Function : Binary Crossentropy


Optimizer : Adam

Not normalize or dropout anything. Just scale image from (0,255) to (0,1)

Learning Rate : 0.1

When apply those, we get accuracy = 0.168457

Lets see what will happened changing each feature one by one





## Decreasing Batch Size

First at all, computational efficiency has increased but we know model has become more regularized & generalized. So we get higher accuracy, and also we could get less test error if I would be  measuring that too . (But unfortunately I have forgot to add it to the table. Because I don't have much time, I couldn't do the same process for to calculate each for test data and compare. But we all know that it will have more generalized (smiling in pain). Whatever, lets go on other results)

Referrence Model
0.1684 -> 0.1687

Number of Hidden Layer  = 2
0.169 >	0.167

Dropout & Normalized
0.17089 ->	0.17083


Final Comment : Getting more generalized and generally get less test error but maybe less accuracy

## Relu ,Tanh and Sigmoid as Activation Function

1 Hidden Layer

relu:	0.168750
tanh:	0.165527
sigmoid: 0.164062

2 Hidden Layer

relu : 0.167708
tanh : 0.165039
sigmoid	: 0.171387

Dropout & Normalized

relu :	0.170833
tanh : 0.151367
sigmoid :	0.173340


As we can see there is not much differences but we can easily say it will depend more deeply parameters. So we can not make a comment about these values. (In 1 hidden layer relu get more accuracy but sigmoid get more accuracy in 2 hidden layers and apply droput & normalization. Maybe we can basicly say sigmoid get higher accuracy when number of hidden layer increases but we can't say that by looking 2 model.)
But we can say tanh has least accuracy for each model

## Change Loss Function
Binary Cross Entropy -> Mean Squared Error  

1 Hidden Layer
Binary Cross Entropy : 0.167969
Mean Squared Error  : 0.162598


2 Hidden Layer

Binary Cross Entropy : 0.171387
Mean Squared Error  : 0.168457

Dropout & Normalized

Binary Cross Entropy : 0.167969
Mean Squared Error  : 0.164551

There no need to more comment on that, Binary Croos Entropy has better accuracy that MSE for our model




## Change Optimizer

Adam -> RMSProp  

1 Hidden Layer
Adam: 0.162598
RMSprop: 0.163574


2 Hidden Layer

Adam: 	0.168457
RMSprop: 	0.163574

Dropout & Normalized

Adam: 0.167969
RMSprop: 0.168457

Need more model comparison to decide which optimizer is better. But accuracy in RMSProp doesn't




## Normalized & Dropout

When applying batch normalization to a network, it  improve training stability and speed. They normalize the inputs to a layer, helping in reducing internal covariate shift. And dropout is a regularization technique where randomly selected neurons are ignored during training. It helps prevent overfitting by introducing noise and promoting robustness.

When we apply these two methods on network we will increase robustness of NN (Normalization dropout can enhance the model's robustness and prevent overfitting) make NN more regularized, make calcultaions faster,
and increase the complexity of the model. So that we need to get more accuracy values.


When we compare rows between 0-7, and 16-23 accuracies (The only difference between these two is 16-23 are a dropped and batch normalized version of first 0-7 respectively  )

## Reducing Learning Rate

0.1->  0.001  

Row ->       LR  |     ACC
***************************
5th row ->	0.100	| 0.163574

7th row ->	0.001	| 0.165527
***************************
13th row ->	0.100	| 0.166504

15th row ->	0.001	| 0.163574
***************************
21th row ->	0.100	| 0.173340

23th row ->	0.001	| 0.168457

We can easily say acc is increasing when decrease learning rate. By reducing the learning rate, the model takes smaller steps in weight updates, fostering a more cautious convergence that helps prevent overshooting the minimum of the loss function. This measured approach improves the stability of training, smoothing the convergence trajectory and preventing potential divergence that might arise with higher learning rates. Lowering the learning rate is particularly advantageous during later stages of training, facilitating fine-tuning and acting as a form of implicit regularization, which aids in preventing overfitting. However, this cautious strategy comes at the cost of increased computational demands, as more iterations may be needed for convergence. In essence, the choice of learning rate is a critical aspect of training neural networks, and experimentation with different rates, along with learning rate schedules, is essential for achieving optimal performance tailored to the specific problem, dataset, and model architecture.



## The reason why is accuracy low
First reason is I have selected random variables for features. For example my data is not binary but I have chose binary cross entropy etc. The reason that why I chose these variables randomly is, if I choose these values by depending on a reason, maybe I couldn't indicate the differences between these values. I know still I couldn't inspect reasons detailly, but the reason for thay is I don't have enough computational speed or enough knowledge about all parameters. But if we chose variables by a reason (For example applying softmax, the accuracy must increase).

The best accuracy whe have is, becomes when we apply  normalization and drop out to network. And best parameters for that is given below

**********************************
19th row

Batch Size : 256

Activation Function : sigmoid

Loss Function : Binary Crossentropy

Optimizer : Adam

Learning Rate:	0.100

Accuracy : 0.173340
**********************************

We include these values with epoch = 10. If I increase this value to 100 and change variable with more reasonable ones, I will probably get more accurate, stable, and applicable model for real life. Also may be a consoltancy about MEL spectgram will be beneficial about improving accuracy of model




# Comparison of MLP & CNN







The superiority of Convolutional Neural Networks (CNNs) over Multi-Layer Perceptrons (MLPs) for certain tasks can be attributed to the unique characteristics of CNNs that make them well-suited for handling images. Because we are handling certain situation considering images features. Below I have listed the reasons why CNN works better:


Local Feature Learning:
We use methods such as convolution and pooling for CNN. Since these methods are tried and tested on images and allow us to process the image better, we achieved better results in CNN.


Architecture of MLP:
There was lot of unexpected results in MLP, so maybe I could created MLP structure not efficient or wrong


Spatial Hierarchies in Data:
CNN combines local patterns (such as edges, textures) in images to form higher-level features (such as shapes and objects). It achieves better results due to the special methods it uses for this purpose.


Parameter Sharing:
CNNs use parameter sharing through convolutional filters, which significantly reduces the number of parameters compared to fully connected layers in MLPs. This makes CNN easier to generalize.


Pooling Layers:
CNNs typically include pooling layers (e.g., max pooling) to subsample spatial dimensions, preserving important features while reducing computation. This hierarchical downsampled representation helps create more abstract and immutable features.


In summary, the architectural design of CNNs, which includes convolutional
layers, pooling layers, and parameter sharing, makes them well-suited for tasks involving grid-like data such as images. MLPs, being fully connected, may struggle to capture spatial hierarchies and translational invariance, leading to inferior performance on image-related tasks. However, the choice of the model also depends on the specific characteristics of your data and the nature of the task at hand.


